## 10. Init

In [1]:
import os
from dotenv import load_dotenv
from pymongo import MongoClient
import hashlib

# Define constants ----------------------------------
load_dotenv()
mongoClient = MongoClient(os.environ.get('MONGO_URI_PV'))
database = mongoClient.pv_data_db
coll_ausgaben = database.ausgaben
coll_artikel = database.artikel

## 20. Import PDF-files, extract text, save to txt-file.

In [ ]:
import pymupdf as fitz
import os

# Ordnerpfad
pdf_folder = 'ausgaben'
text_folder = 'ausgaben_txt'

# Erstellen Sie den Ordner für die Textdateien, falls er nicht existiert
if not os.path.exists(text_folder):
    os.makedirs(text_folder)

# Durchlaufen Sie alle PDF-Dateien im Ordner
for filename in os.listdir(pdf_folder):
    if filename.endswith('.pdf'):
        pdf_path = os.path.join(pdf_folder, filename)
        text_path = os.path.join(text_folder, f"{os.path.splitext(filename)[0]}.txt")
        
        # PDF-Datei öffnen
        pdf_document = fitz.open(pdf_path)
        
        # Text aus der PDF-Datei extrahieren
        text = ""
        for page_num in range(pdf_document.page_count):
            page = pdf_document.load_page(page_num)
            text += page.get_text()
        
        # Text in eine Textdatei schreiben
        with open(text_path, 'w', encoding='utf-8') as text_file:
            text_file.write(text)

print("Text aus allen PDF-Dateien wurde extrahiert und gespeichert.")

Text aus allen PDF-Dateien wurde extrahiert und gespeichert.


## 25. Test pypdf & visitors

In [3]:
import pypdf

parts = []

def visitor_body(text, cm, tm, font_dict, font_size):
    y = cm[5]              # y coordinate
    header_max_y = 900     # header
    footer_min_y = 85     # footer
    if footer_min_y < y < header_max_y:
        parts.append(text)

file_name = 'ausgaben/pvd Ausgabe #11 2024.pdf'
pdf_doc = pypdf.PdfReader(file_name)

print(f"Seiten: {len(pdf_doc.pages)}")

text = pdf_doc.pages[3].extract_text()
        
# for page in pdf_doc.pages:
pdf_doc.pages[3].extract_text(visitor_text=visitor_body)

text_visitor = "".join(parts)

print("-" * 50)
print(text)
print("-" * 50)
print(f"Parts: {len(parts)}")
print(parts)
print("-" * 50)
print(text_visitor)
print("-" * 50)


Seiten: 31
--------------------------------------------------
 
ZEITUNGEN, ZEITSCHRIFTEN, PAID CONTENT 
INFORMATIONEN FÜR DEN VERLEGERISCHEN ERFOLG IM LESERMARKT  
 
 
 
 
STOP!  Bitte stellen Sie sicher, dass Sie das sind. Wenn nicht, dann lesen Sie in einer unlizenzierten Ausgabe. Das gefährdet das Erscheinen von pv digest! Für 
mehrere Leser innerhalb eines Unternehmens haben wir attraktive Konditionen. Sprechen Sie uns dazu gerne an: abo@pv-digest.de / Tel.: 040 / 308 901 09. 
AUSGABE 11/2024 ׀SEITE 4 
 
Zwei Jahre L'Equipe zum Schnäppchenpreis – und dann? 
Mitte Oktober veranstaltete der deutsche (Digital -)Abomanagement-Dienstleister Plenigo 
seinen 'Community Day'. Eine Sprecherin auf der sehr gut besuchten Veranstaltung war die 
Mitgründerin des französischen Paywallmanagement- Dienstleisters Poool [pvd: hier bahnt 
sich vielleicht eine sehr spannende Allianz an. Plenigo würde in Frankreich einen großen Markt 
vorfinden – Poools Ansatz bei der Paywallsteuerung (vgl. auch  
S. 2

## 30. Import Ausgaben im TXT Format

In [ ]:
text_folder = "ausgaben_txt"

for filename in os.listdir(text_folder):
    # Open file and read content
    with open(f"{text_folder}/{filename}", "r") as file:
        content = file.read()

    # Create doknr
    ausgabe = filename[:-4].split(" ")[2][1:3]
    jahrgang = filename[:-4].split(" ")[3]
    doknr = f"{jahrgang}.{ausgabe}"

    # Check if hash already exists in database
    if coll_ausgaben.find_one({"doknr": doknr}):
        print(f"File {filename} already exists in database")
    else:
        # Insert new document into database
        coll_ausgaben.insert_one({
            "doknr": doknr,
            "jahrgang": int(jahrgang),
            "ausgabe": int(ausgabe),
            "text": content,
            })
        print(f"File {filename} added to database")

## 40. In Chunks zerlegen

In [5]:
def chunk_text_to_dataframe(text:str, chunk_size:int, overlap:int=0) -> list:
    """
    Splits a text into chunks and stores them in a Pandas DataFrame.

    Args:
        text: The input text string.
        chunk_size: The desired size of each chunk (number of characters).
        overlap: The number of overlapping characters between chunks.  Defaults to 0 (no overlap).

    Returns:
        A Pandas DataFrame where each row represents a chunk of text.  Returns an empty DataFrame if the input text is None or empty.
        Returns None if chunk_size is invalid (<= 0) or overlap is negative or greater than or equal to chunk_size.
    """

    if not text:  # Handle None or empty input
        return pd.DataFrame()

    if chunk_size <= 0:
      return None

    if overlap < 0 or overlap >= chunk_size:
      return None

    chunks = []
    start = 0
    while start < len(text):
        end = min(start + chunk_size, len(text))  # Ensure we don't go past the end of the text
        chunk = {'start': start, 'end': end, 'text': text[start:end]}
        chunks.append(chunk)
        start += chunk_size - overlap # Move start for the next chunk, accounting for overlap

    return chunks

## 50. Chunks in Collection ARTIKEL einfügen

In [ ]:
cursor = coll_ausgaben.find()
for ausgabe in cursor:
    if coll_artikel.find_one({"doknr": ausgabe["doknr"]}):
        print(f"Document {ausgabe["doknr"]} already exists in database")
    else:
        chunks = chunk_text_to_dataframe(ausgabe['text'], 1000, 200)
        for chunk in chunks:
            coll_artikel.insert_one({
                "doknr": ausgabe['doknr'],
                "start": chunk['start'],
                "end": chunk['end'],
                "text": chunk['text'],
                "embeddings": [],
            })


In [ ]:
cursor = coll_artikel.find()
for artikel in cursor:
    print(f"{artikel['doknr']} [{artikel['start']}-{artikel['end']}]")


## 60. Embeddings in Collection ARTIKEL generieren

In [3]:
import os
import torch
from transformers import BertTokenizer, BertModel

# Load pre-trained model and tokenizer
os.environ["TOKENIZERS_PARALLELISM"] = "false"
model_name = "bert-base-german-cased" # 768 dimensions
# model_name = "bert-base-multilingual-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# Embeddings -------------------------------------------------            
def create_embeddings(text: str) -> list:
    encoded_input = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        model_output = model(**encoded_input)
    return model_output.last_hidden_state.mean(dim=1).squeeze().tolist()

def generate_embeddings(input_field: str, output_field: str, 
                        max_iterations: int = 10) -> None:
    cursor = coll_artikel.find({output_field: []}) #.limit(max_iterations)
    cursor_list = list(cursor)
    for record in cursor_list:
        article_text = record[input_field]
        if article_text == "":
            article_text = "Fehler: Kein Text vorhanden."
        else:
            embeddings = create_embeddings(text=article_text)
            coll_artikel.update_one({"_id": record['_id']}, {"$set": {output_field: embeddings}})
    print(f"\nGenerated embeddings for {max_iterations} records.")

In [4]:
generate_embeddings("text", "embeddings")


Generated embeddings for 10 records.
